In [0]:
import pandas as pd
import tensorflow as tf
import os
import re
import numpy as np
from string import punctuation
from zipfile import ZipFile
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
PATH='/content/gdrive/My Drive/ML Project/test.csv'

In [4]:
data = pd.read_csv(PATH)
print('총 샘플의 수 :',len(data))
train_data=data.copy()
train_data['title'].nunique(), data['text'].nunique() #기사제목, 기사내용 중복 내용 확인
# 윈도우 바탕화면에서 작업한 저자의 경우에는
# data = pd.read_csv(r'C:\Users\USER\Desktop\spam.csv',encoding='latin1')
train_data.drop_duplicates(subset=['text'], inplace=True) # v2 열에서 중복인 내용이 있다면 중복 제거
print('중복 제거 후 샘플의 수 :',len(train_data))

총 샘플의 수 : 5200
중복 제거 후 샘플의 수 : 5127


In [0]:
# importing neural network libraries
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Embedding, GRU, LSTM, RNN, SpatialDropout1D
from tensorflow.keras.models import load_model

In [0]:
train_data = train_data.set_index('id', drop = True)

In [7]:
print(train_data.shape)
train_data.head()

(5127, 3)


,title,author,text
id,,,
20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning..."
20801,Russian warships ready to strike terrorists ne...,NaN,Russian warships ready to strike terrorists ne...
20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...
20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different..."
20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...


In [8]:
train_data.isnull().sum()

title     120
author    490
text        1
dtype: int64

In [9]:
train_data[['title', 'author']] = train_data[['title', 'author']].fillna(value = 'Missing')
train_data = train_data.dropna()
train_data.isnull().sum()

title     0
author    0
text      0
dtype: int64

In [10]:
length = []
[length.append(len(str(text))) for text in train_data['text']]
train_data['length'] = length
train_data.head()

,title,author,text,length
id,,,,
20800,"Specter of Trump Loosens Tongues, if Not Purse...",David Streitfeld,"PALO ALTO, Calif. — After years of scorning...",8015
20801,Russian warships ready to strike terrorists ne...,Missing,Russian warships ready to strike terrorists ne...,1559
20802,#NoDAPL: Native American Leaders Vow to Stay A...,Common Dreams,Videos #NoDAPL: Native American Leaders Vow to...,4547
20803,"Tim Tebow Will Attempt Another Comeback, This ...",Daniel Victor,"If at first you don’t succeed, try a different...",3578
20804,Keiser Report: Meme Wars (E995),Truth Broadcast Network,42 mins ago 1 Views 0 Comments 0 Likes 'For th...,542


In [11]:
train_data = train_data.drop(train_data['text'][train_data['length'] < 50].index, axis = 0)
print('기사의 최대길이 : %d' % max(j for j in train_data['length']))

기사의 최대길이 : 107014


In [0]:
max_features = 4500

In [0]:
tokenizer = Tokenizer(num_words = max_features, filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n', lower = True, split = ' ')
tokenizer.fit_on_texts(texts = train_data['text'])
X = tokenizer.texts_to_sequences(texts = train_data['text'])

In [14]:
print(X[2000])

[2493, 3970, 3, 5, 668, 3, 908, 2406, 173, 177, 68, 300, 77, 4, 25, 20, 464, 6, 49, 3, 1, 24, 4414, 77, 112, 1239, 3, 8, 827, 2, 3330, 104, 764, 7, 16, 35, 48, 6, 571, 5, 87, 415, 714, 31, 368, 256, 56, 2313, 2, 125, 40, 14, 38, 65, 136, 67, 11, 16, 264, 958, 1100, 300, 77, 2354, 1318, 6, 3, 884, 300, 77, 2628, 24, 2287, 2235, 33, 4326, 1221, 3656, 37, 19, 733, 1, 3569, 12, 530, 507, 10, 390, 19, 1, 76, 3, 1, 3569, 1, 144, 35, 43, 1457, 83, 611, 40, 458, 4115, 13, 4, 46, 12, 1892, 7, 1, 3, 611, 82, 568, 2, 72, 90, 46, 48, 415, 15, 29, 1668, 9, 1, 2167, 5, 3599, 12, 113, 1442, 18, 29, 3060, 511, 33, 965, 7, 910, 13, 77, 1219, 48, 24, 571, 1, 415, 53, 1, 511, 1021, 46, 12, 47, 6, 1, 3970, 77, 8, 827, 10, 90, 341, 2, 392, 13, 1, 511, 4, 611, 7, 117, 1, 3, 1, 3970, 2813, 13, 1, 87, 415, 44, 388, 2, 1, 103, 144, 6, 4182, 28, 276, 74, 49, 145, 63, 77, 35, 748, 10, 1518, 1012, 13, 25, 860, 120, 34, 1, 546, 258, 233, 887, 34, 104, 910, 13, 306, 6, 25, 9, 5, 976, 3561, 1787, 1442, 105, 81, 18, 

In [15]:
X = pad_sequences(sequences = X, maxlen = max_features, padding = 'pre')
print(X.shape)

(5093, 4500)


In [28]:
import numpy as np
loaded_model = load_model('/content/gdrive/My Drive/ML Project/GRUbest_model.h5')
##xhat = X[1]
##yhat = loaded_model.predict(xhat)
##print('Predict : ' + str(yhat))

xhat_idx = np.random.choice(X.shape[0], 5)
xhat = X[xhat_idx]
yhat = loaded_model.predict_classes(xhat)

for i in range(5):
    if(yhat[i]==0):
      print('Predict : True '+ str(yhat[i]))
    else:
      print('Predict : False '+ str(yhat[i]))
    ##print('Predict : ' + str(yhat[i]))

Predict : False [1]
Predict : False [1]
Predict : False [1]
Predict : True [0]
Predict : False [1]
